[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RonPlusSign/AnomalySegmentation/blob/chris-1/Project6.ipynb)

# **Anomaly Segmentation Project 6**
##*Andrea Delli, Christian Dellisanti, Giorgia Modi*

###**Dataset Preparation**

In [1]:
'''
SMIYC RA-21
if not os.path.isfile('/content/dataset_AnomalyTrack.zip'):
  !wget https://zenodo.org/record/5270237/files/dataset_AnomalyTrack.zip
if not os.path.isdir('/content/dataset_AnomalyTrack'):
  !unzip ./dataset_AnomalyTrack.zip
#SMIYC RO-21
if not os.path.isfile('/content/dataset_ObstacleTrack.zip'):
  !wget https://zenodo.org/record/5281633/files/dataset_ObstacleTrack.zip
if not os.path.isdir('/content/dataset_ObstacleTrack'):
  !unzip ./dataset_ObstacleTrack.zip

#FS L&F
if not os.path.isfile('/content/fishyscapes_lostandfound.zip'):
  !wget http://robotics.ethz.ch/~asl-datasets/Fishyscapes/fishyscapes_lostandfound.zip
if not os.path.isdir('/content/fishyscapes_lostandfound'):
  !unzip ./fishyscapes_lostandfound.zip -d fishyscapes_lostandfound
'''

!pip  install numpy matplotlib Pillow torchvision visdom ood_metrics

import sys, os
if not os.path.isfile('/content/Validation_Dataset.zip'):
  !gdown 12YJq48XkCxQHjN3CmLc-zM5dThSak4Ta
if not os.path.isdir('/content/Validation_Dataset'):
  !unzip Validation_Dataset.zip
if not os.path.isdir('/content/AnomalySegmentation_CourseProjectBaseCode'):
  #!git clone https://github.com/shyam671/AnomalySegmentation_CourseProjectBaseCode.git
  !git clone -b chris-1 https://github.com/RonPlusSign/AnomalySegmentation.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 36.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408196 sha256=62fe661397b5f5155da3a92cef3725416e7f4ba5124e07502c8fd4e1d9023dd7
  Stored in directory: /root/.cache/pip/wheels/42/29/49/5bed207bac4578e4d2c0c5fc0226bfd33a7e2953ea56356855
Successfully built visdom
Downloading...
From (original): https://drive.google.com/uc?id=12YJq48XkCxQHjN3CmLc-zM5dThSak4Ta
From (redirected): https://drive.google.com/uc?id=12YJq48XkCxQHjN3CmLc-zM5dThSak4Ta&confirm=t&uuid=b06589e5-4c0f-4043-a2c3-b7cd8a947521
To: /content/Validation_Dataset.zip
100% 329M/329M [00:11<00:00, 29.6MB/s]
Archive:  Validation_Dataset.zip
   creating: Validation_Dataset/
   creating: Validation_Dataset/RoadAnomaly21/
  inflating: Validation_Dataset/.DS_Store  
  inflating: __MACOSX/Validation_Dataset/._.DS_Store  
   creating: Validation_Dataset/RoadAnomaly/
   creating: Validation_Datase

###**Anomaly Inference**

In [1]:
import torch
# se vado alla linea 87 di evalAnomaly.py quello dovrebbe essre l'utilizzo di MSP
if torch.cuda.is_available():
  !python /content/AnomalySegmentation_CourseProjectBaseCode/eval/evalAnomaly.py --input '/content/Validation_Dataset/FS_LostFound_full/images/*.png' --loadDir '/content/AnomalySegmentation_CourseProjectBaseCode/trained_models/'

FileNotFoundError: [Errno 2] No such file or directory: '/content/AnomalySegmentation_CourseProjectBaseCode'

**DATASET MANAGEMENT** **TUTTO INUTILE GIÀ TUTTO SCRITTO**

In [ ]:
from torch.utils.data import Dataset
import os

from PIL import Image
import sys


def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(f"/content/Validation_Dataset/{path}", 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')


# Define the Dataset class
class PACSDataset(Dataset):
    def __init__(self, root= "/content/Validation_Dataset", source_domain="FS_LostFound_full" , dataset_transform=None):
      super(PACSDataset, self).__init__( )

      self.dataset_transform = dataset_transform

      self.root=f"{root}/{source_domain}"
      self.data   = os.listdir(f"{root}/{source_domain}/images")
      #self.labels = os.listdir(f"{root}/{source_domain}/labels_masks")



    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):

      image, label = pil_loader(f"images/{self.data[index]}"), pil_loader(f"labels_masks/{self.data[index]}")

      # Applies preprocessing when accessing the image
      if self.dataset_transform is not None:
          image = self.dataset_transform(image)

      return image, label






**ARCHITECTURE SETUP**

In [ ]:
import torch.nn as nn

# Define AlexNet architecture class
class ErfNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        # Category classifier
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes)
        )
        # Domain classifier
        self.domain_classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 2)
        )

    def forward(self, x):
        features = self.features(x)
        features = features.view(features.size(0), -1)
        class_outputs = self.classifier(features)
        domain_outputs = self.domain_classifier(features)
        return class_outputs, domain_outputs